In [1]:
load("../Imports/GeneralMethods.sage", '../Imports/Utility.sage', '../Imports/Point.sage', '../Imports/Line.sage', '../Imports/TritangentPlane.sage', '../Imports/Group.sage', '../Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
xx = QQ['xx'].0
Q.<rad5> = NumberField(xx^2-5)

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [4]:
# Define cubic SE10, find and classify the lines and compute the 45 tritangent planes

In [5]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE10 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f:c*(3*c-e)/(c+e)}).subs({e : (2-rad5)*c})

In [6]:
# Find simmetries

In [7]:
%time adm_SE10 = SE10.find_admissible_projectivities()
len(adm_SE10)

CPU times: user 796 ms, sys: 76.2 ms, total: 872 ms
Wall time: 3.17 s


120

In [8]:
%time simm_SE10 = SE10.find_simmetries(adm_SE10)
len(simm_SE10)

CPU times: user 538 ms, sys: 84.5 ms, total: 622 ms
Wall time: 823 ms


120

In [9]:
# Study how the simmetries permute the Eckardt points

In [10]:
Eck_perms = [Permutation(SE10.apply_proj_to_eck(proj[0])).to_permutation_group_element() for proj in simm_SE10]
Eck_perms_group = PermutationGroup(Eck_perms)
Eck_perms_group.order(), Eck_perms_group.structure_description()

(120, 'S5')

In [11]:
### geometric interpretation of the group of symmetries

In [12]:
## Action of the stabilizer of SE10 on the 27 lines:
Ga = perm_group_lines(SE10, simm_SE10)

## Action of the stabilizer of SE10 on the 45 tritangent planes:
Gb = perm_group_planes(SE10, simm_SE10)

print("Structure of the group Ga:")
print(PermutationGroup(Ga).structure_description())

print("")

print("Structure of the group Gb:")
print(PermutationGroup(Gb).structure_description())

Structure of the group Ga:
S5

Structure of the group Gb:
S5


In [13]:
## Action of the group Ga on the lines: computation of the orbits:
OGa = PermutationGroup(Ga).orbits()

## Action of the group on Gb the planes: computation of the orbits:
OGb = PermutationGroup(Gb).orbits()

In [14]:
## The orbits OGb are a partition of the set {1, 2, ..., 45}:
print(OGb)

[[1, 3, 7, 8, 11, 12, 16, 18, 31, 34], [2, 6, 17, 13, 37], [4, 32, 27, 5, 29, 39, 23, 9, 43, 10, 24, 44, 35, 21, 15, 14, 22, 40, 26, 20, 38, 28, 45, 19, 33, 42, 30, 36, 41, 25]]


In [15]:
for orb in OGb:
    print([simbolic_plane_from_index(i) for i in orb])

['t1', 't3', 't7', 't8', 't11', 't12', 't16', 't18', 't31', 't34']
['t2', 't6', 't17', 't13', 't37']
['t4', 't32', 't27', 't5', 't29', 't39', 't23', 't9', 't43', 't10', 't24', 't44', 't35', 't21', 't15', 't14', 't22', 't40', 't26', 't20', 't38', 't28', 't45', 't19', 't33', 't42', 't30', 't36', 't41', 't25']


In [16]:
## The group Gb permutes the elements of the second orbit of OGb (which has 5 elements)
## in all the 5! possibilities:
O1 = OGb[1]
totPerm = []
for gg in Gb:
    totPerm.append(tuple([gg(i) for i in O1]))

In [17]:
len(Set(totPerm))

120

In [18]:
### hence OGb is S_4.

In [ ]:
# SE10 is the sum of the cubes of the following planes

In [36]:
planes = [SE10.tritangent_planes[i-1].plane for i in OGb[1]]
planes

[y - z,
 (rad5 - 3)*x + (rad5 - 3)*t,
 (rad5 - 3)*x + (-2*rad5 + 6)*y + (rad5 - 3)*t,
 x - y,
 -2*x + y - z]

In [57]:
Sylvester_planes = [planes[0], -planes[1]/(rad5 - 3), planes[2]/(rad5 - 3), -2*planes[3], -planes[4]]
Sylvester_planes

[y - z, -x - t, x - 2*y + t, -2*x + 2*y, 2*x - y + z]

In [58]:
nc = sum([plane^3 for plane in Sylvester_planes])
nc/6 == SE10.eqn

True